# B. Sim Dashboard

### Imports

In [ ]:
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [ ]:
# Schedule
pause_code(start_time='2024-06-06T12:25:00', timezone='EST')

### Games

Select start and end date (should be todaysdate for both if running day-of)

In [ ]:
# start_date = "20240420"
start_date = todaysdate
end_date = todaysdate

Read in games

In [ ]:
game_df = read_and_save_games(team_map, generate=True)
game_df = game_df[(game_df['date'] >= start_date) & (game_df['date'] <= end_date)].reset_index(drop=True)

### Settings

In [ ]:
# Run historic matchups
historic = False
# Run matchups necessary for slate
slate_only = True

# Run:
Matchups = True
Simulations = True
Optimizer = True

In [ ]:
%%time
if historic == False:
    %run "A. Night Dashboard.ipynb"

### Contest

In [ ]:
# Read in subset of contests for which we'll create guides
subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {todaysdate}.csv'))
subset_df['contestDate'] = pd.to_datetime(subset_df['contestDate'])
four_seamer_df = subset_df[(subset_df['Name'].str.contains('eamer')) & ~(subset_df['Name'].str.contains('Turbo'))].sort_values('contestDate', ascending=True).reset_index(drop=True)
four_seamer_df

In [ ]:
# Set contestKey
contestKey = four_seamer_df['contestKey'][3]

# Read in Contest Guide
guide = pd.read_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"))

# Identify draftGroupId
draftGroupId = guide['draftGroupId'][0]

# Identify RotoWire slate
roto_slate = guide['roto_slate'][0]

# Print out name
print(f"Date: {guide['date'][0]}. Slate: {guide['slate'][0]}. {guide['name'][0]}. {roto_slate}.")

In [ ]:
if slate_only == True:
    slate_df = game_df[game_df['game_id'].isin(guide['game_id'].unique())].reset_index(drop=True)

### Weather

In [ ]:
try:
    rotogrinders_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "2. RotoGrinders", f"RotoGrinders {todaysdate}.csv"))
    red_list = list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Away']) + list(rotogrinders_df.query('Tag == "red" or Tag2 == "red"')['Home'])
    orange_list = list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Away']) + list(rotogrinders_df.query('Tag == "orange" or Tag2 == "orange"')['Home'])
    display_weather = rotogrinders_df[['Tag', 'Tag2', 'Away', 'Home', 'date', 'Description']]
except:
    red_list = []
    orange_list = []
    display_weather = None
    
display_weather

### B01. Matchups

In [ ]:
%run "C:\Users\james\Documents\MLB\Code\B01. Matchups.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A02. MLB API.ipynb"
%run "C:\Users\james\Documents\MLB\Code\A03. Steamer.ipynb"

##### Load
This loads in datasets needed to create matchup files

In [ ]:
%%time
complete_dataset = pd.read_csv(os.path.join(baseball_path, "Stat Dataset.csv"))
steamer_hitters_df = pd.read_csv(os.path.join(baseball_path, "Steamer Hitters.csv"))
steamer_pitchers_df = pd.read_csv(os.path.join(baseball_path, "Steamer Pitchers.csv"))

Create matchups

In [ ]:
%%time
try:
    print(len(game_df))
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))
except:
    print("Trying Again")
    empty_list = Parallel(n_jobs=-1, verbose=True)(delayed(create_matchup_files)(game_df, row, complete_dataset, steamer_hitters_df, steamer_pitchers_df, team_map) for row in range(len(game_df)))

### B02. Simulations

In [ ]:
%run "C:\Users\james\Documents\MLB\Code\B02. Simulations.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
%run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"

In [ ]:
historic, slate_only

In [ ]:
num_batches = int(os.cpu_count() / 1)
batch_size = int(63 * 1)

In [ ]:
if historic == False:
    # Openers
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f"Draftables {draftGroupId}.csv"), encoding='iso-8859-1')
    opener_list = list(draftables.query('Salary < 5000 and `Roster Position` == "P"')['Name'])
    print(f"Openers: {opener_list[:10]}")

    # Teams with Projected Lineups
    projected_lineups = pd.read_csv(os.path.join(baseball_path, "A05. Rosters", "3. Batting Orders Projected", f"Batting Orders Projected {todaysdate}.csv"))
    projected_teams = projected_lineups.merge(team_map[['SFBBTEAM', 'DKTEAM']], left_on=['team code'], right_on=['SFBBTEAM'], how='inner')
    projected_team_list = list(projected_teams.query('` confirmed` == "N"')['DKTEAM'].unique())
    print(f"Projected teams: {projected_team_list}")
else:
    opener_list = []
    projected_lineups = None
    projected_team_list = []

In [ ]:
# Choose which games to run simulations for
if slate_only == True:
    # Just the slate
    run_df = slate_df.copy()
else:
    # All games
    run_df = game_df.copy()

In [ ]:
%%time
# Print out games to simulate
_ = [print(f"{row['away_team']}@{row['home_team']} {pd.to_datetime(row['game_datetime']).tz_convert('US/Eastern').strftime('%Y-%m-%d %H:%M:%S')}") for index, row in run_df.head(15).iterrows()]

# Loop over dates of interest
for date in list(run_df['date'].unique()):
    print(date)
    
    # Extract year
    year = date[:4]
    # Extract matchups
    matchup_list = [os.path.splitext(f)[0] for f in os.listdir(os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}"))]
    
    # Loop over games
    daily_df = run_df.query(f'date == "{date}"').reset_index()
    print(daily_df.shape)
    for i in range(len(daily_df)):            
        # Extract info from run_df to look up proper file in matchups path
        away_team = daily_df['away_team'][i]
        home_team = daily_df['home_team'][i]
        game_id = daily_df['game_id'][i]
        # Beginning of filename in matchup folder
        lookup = f"{away_team}@{home_team} {game_id}"
        
        # Find the matchup file
        matchup = next((matchup for matchup in matchup_list if matchup.startswith(lookup)), None)
        print(matchup)
        
        # Create matchups
        matchup_path = os.path.join(baseball_path, "B01. Matchups", f"Matchups {date}", f"{matchup}.xlsx")
        AwayBatters, HomeBatters, AwayPitchers, HomePitchers = create_matchup(matchup_path, batter_stats_scaler, batter_stats_fg_scaler, batter_imputations_model, pitcher_stats_scaler, pitcher_stats_fg_scaler, pitcher_imputations_model, projected_lineups)

        # Read in weather
        weather_df = pd.read_csv(os.path.join(baseball_path, "A06. Weather", "4. Park and Weather Factors", f"Park and Weather Factors {date}.csv"))
        weather_df = weather_df.query(f'game_id == {game_id}').reset_index(drop=True)
        
        # Create Scoreboard object
        game_template = Scoreboard(AwayBatters, HomeBatters, AwayPitchers, HomePitchers, 9)       
        
        ### Sim games
        # With batching (fast!)
        start = time.time()
        game_list = Parallel(n_jobs=num_batches, verbose=1)(delayed(sim_game_batch)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=False, batch_size=batch_size) for batches in range(num_batches))
        game_list = [game for sublist in game_list for game in sublist]
        print(f"Simming {batch_size*num_batches} games took {time.time() - start} seconds.")
        
        # Without batching (slow!)
        # game_list = Parallel(n_jobs=16, verbose=0)(delayed(sim_game)(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=False) for sims in range(num_sims))
        
        # # Without joblib (slower!)
        # start = time.time()
        # game_list = []
        # for i in range(10):
        #     game = sim_game(game_template, model_pulls, model_binary, model_outs, model_safe, opener_list, weather_df, year, innings=9, debug=True)
        #     game_list.append(game)
        # print(f"Simming {num_sims} games took {time.time() - start} seconds.")
        
        # Create object path
        game_path = os.path.join(baseball_path, "B02. Simulations", "1. Game Sims", f"Matchups {date}", f"{matchup}") 
        player_path = os.path.join(baseball_path, "B02. Simulations", "2. Player Sims", f"Matchups {date}", f"{matchup}") 
        # Clear folders
        if os.path.exists(game_path):
            shutil.rmtree(game_path)
        if os.path.exists(player_path):
            shutil.rmtree(player_path)
        # Make folders
        os.makedirs(game_path, exist_ok=True)
        os.makedirs(player_path, exist_ok=True)
        
        # Save each object in the list to a separate file
        for i, game_object in enumerate(game_list):
            # Delete unnecessary attributes from batter objects
            for batter in game_object.away_batters + game_object.home_batters:
                batter = batter.keep_selected_attributes()
            # Delete unnecessary attributes from pitcher objects
            for pitcher in game_object.away_pitchers + game_object.home_pitchers:
                pitcher = pitcher.keep_selected_attributes()
            # Delete unnecessary attributes from game objects
            game_object.keep_selected_attributes()

            # Constructing DataFrames for game, batters, and pitchers
            game_score_df = pd.DataFrame({
                "away_score": [game_object.away_score],
                "home_score": [game_object.home_score]
            })

            away_batters_df = pd.DataFrame([vars(batter) for batter in game_object.away_batters])
            away_batters_df['team'] = "away"
            home_batters_df = pd.DataFrame([vars(batter) for batter in game_object.home_batters])
            home_batters_df['team'] = "home"
            batters_df = pd.concat([away_batters_df, home_batters_df], axis=0)
            
            away_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.away_pitchers])
            away_pitchers_df['team'] = "away"
            home_pitchers_df = pd.DataFrame([vars(pitcher) for pitcher in game_object.home_pitchers])
            home_pitchers_df['team'] = "home"
            pitchers_df = pd.concat([away_pitchers_df, home_pitchers_df], axis=0)
            
            # Specify the file paths where you want to save the CSV files
            game_file_path = os.path.join(game_path, "game_{}.csv").format(i)
            batters_file_path = os.path.join(player_path, "batters_{}.csv").format(i)
            pitchers_file_path = os.path.join(player_path, "pitchers_{}.csv").format(i)

            # Saving DataFrames to CSV files
            game_score_df.to_csv(game_file_path, index=False)
            batters_df.to_csv(batters_file_path, index=False)
            pitchers_df.to_csv(pitchers_file_path, index=False)

        print(f'Saved to {game_path}')

In [ ]:
# Optionally, read in C02, vegas scores, my scores

### B03. Lineups

In [ ]:
%run "U1. Imports.ipynb"
%run "U2. Utilities.ipynb"
%run "C02. Projections.ipynb"
%run "B03. Optimizer.ipynb"

In [ ]:
sort_by = 'Plus3'
min_salary = 49000
min_projection = (5, 5, 3)
major_stack = (5, 5, 4)
minor_stack = (2, 2, 2)
max_exposure_batters = 0.5
max_exposure_pitchers = 0.7
excluded_teams = ([team for var in ['red_list', 'orange_list', 'projected_team_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]], [team for var in ['red_list'] if globals().get(var) is not None for team in globals()[var]])
min_starters = (10, 10, 0)
lineups = 200
historic= False

### Run One

##### 1. Players

In [ ]:
draftables_with_sims = create_player_file(contestKey, guide, draftGroupId, roto_slate, max_exposure_pitchers, max_exposure_batters)
draftables_with_sims.to_csv(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"), index=False, encoding='iso-8859-1')

In [ ]:
webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "1. Players", f"Players {contestKey}.csv"))

##### 2. Lineups

In [ ]:
%%time
attempts = [
    ("Maximum Constraints", min_projection[0], major_stack[0], minor_stack[0], excluded_teams[0], min_starters[0]),
    ("Some Constraints", min_projection[1], major_stack[1], minor_stack[1], excluded_teams[1], min_starters[1]),
    ("Minimum Constraints", min_projection[2], major_stack[2], minor_stack[2], excluded_teams[2], min_starters[2])
]

for attempt_name, projection, major, minor, excluded, starters in attempts:
    print(f"Attempting: {attempt_name}")
    try:
        create_lineups(contestKey, min_salary, projection, major, minor, excluded, starters, lineups)
        break  # Exit loop if successful
    except Exception as e:
        print(f"Attempt failed: {e}\n")

##### 3. Lineups Ranked

In [ ]:
sort_by_list = ['P50', 'P75', 'P90', 'P95', 'P99', 'P100', 'Tail', 'Sim STD', 'Plus2', 'Plus3', 'rostership', 'pitcher rostership', 'batter_rostership']
lineups_ranked = choose_lineups(contestKey, roto_slate, sort_by)
lineups_ranked.to_csv(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"), index=False)

In [ ]:
# webbrowser.open(os.path.join(baseball_path, "B03. Lineups", "3. Lineups Ranked", f"Lineups Ranked {contestKey}.csv"))

In [ ]:
lineups_ranked.query('pareto == 1')

##### 4. Uploads

In [ ]:
upload = create_upload_file(contestKey, sort_by)
upload.to_csv(os.path.join(baseball_path, "B03. Lineups", "4. Uploads", f"Upload {contestKey}.csv"), index=False)

##### 5. Entries

In [ ]:
entry = create_entry_file(draftGroupId, contestKey)
entry.to_csv(os.path.join(baseball_path, "B03. Lineups", "5. Entries", f"Entries {draftGroupId}.csv"), index=False, encoding='iso-8859-1')

##### Upload

In [ ]:
try:
    upload_entries(draftGroupId)
except:
    upload_entries(draftGroupId)

##### Email

In [ ]:
email_upload_file(draftGroupId, contestKey)

### Run All

In [ ]:
# Create list of contestKeys with available guides
contestKey_list = os.listdir(os.path.join(baseball_path, "A09. Contest Guides"))
contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in contestKey_list]


# Create list of contestKeys with available results

result_contestKey_list = os.listdir(os.path.join(baseball_path, "A01. DraftKings", "6. Player Results"))
result_contestKey_list = [int(key.replace(".csv", "").split(" ")[2]) for key in result_contestKey_list]

# Find the overlap 
available_contestKey_list = list(set(result_contestKey_list) & set(contestKey_list))

In [ ]:
# %%time
# fail_list = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by, min_salary, min_projection[0], major_stack[0], minor_stack[0], excluded_teams=[], lineups, historic=True) for contestKey in available_contestKey_list)

In [ ]:
# %%time
# fail_list2 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=5, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list if contestKey is not None])

In [ ]:
# %%time
# fail_list3 = Parallel(n_jobs=-1, verbose=1)(delayed(create_contest_lineups2)(contestKey, sort_by='Plus3', min_salary=49000, min_projection=4, major_stack=4, minor_stack=2, max_exposure_batters=0.5, max_exposure_pitchers=0.75, excluded_teams=[], lineups=lineups, historic=True) for contestKey in [contestKey for contestKey in fail_list2 if contestKey is not None])